In [7]:
# 1. Library imports
import uvicorn
import gunicorn
from fastapi import FastAPI
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [8]:
data = pd.read_csv('clean_data.csv')

In [9]:
data

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,PREV_PRODUCT_COMBINATION_CashXSellmiddle_MEAN,PREV_PRODUCT_COMBINATION_POShouseholdwithinterest_MEAN,PREV_PRODUCT_COMBINATION_POShouseholdwithoutinterest_MEAN,PREV_PRODUCT_COMBINATION_POSindustrywithinterest_MEAN,PREV_PRODUCT_COMBINATION_POSindustrywithoutinterest_MEAN,PREV_PRODUCT_COMBINATION_POSmobilewithinterest_MEAN,PREV_PRODUCT_COMBINATION_POSmobilewithoutinterest_MEAN,PREV_PRODUCT_COMBINATION_POSotherwithinterest_MEAN,PREV_PRODUCT_COMBINATION_POSotherswithoutinterest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
0,100002,1,0,0,0,0,202500.0,406597.5,0.018801,-9461,...,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
1,100003,0,1,0,1,0,270000.0,1293502.5,0.003541,-16765,...,0.00,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0
2,100004,0,0,1,0,0,67500.0,135000.0,0.010032,-19046,...,0.00,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0
3,100007,0,0,0,0,0,121500.0,513000.0,0.028663,-19932,...,0.50,0.166667,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0
4,100008,0,0,0,0,0,99000.0,490495.5,0.035792,-16941,...,0.20,0.400000,0.0,0.000000,0.0,0.200000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267143,456249,0,1,0,0,0,112500.0,225000.0,0.022800,-24384,...,0.00,0.000000,0.5,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
267144,456251,0,0,0,1,0,157500.0,254700.0,0.032561,-9327,...,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0
267145,456253,0,1,0,0,0,153000.0,677664.0,0.005002,-14966,...,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0
267146,456254,1,1,0,0,0,171000.0,370107.0,0.005313,-11961,...,0.00,0.500000,0.0,0.000000,0.0,0.500000,0.0,0.0,0.0,0.0


In [10]:
X = data.drop(columns='TARGET')
y = data['TARGET']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.1)

In [12]:
model = LGBMClassifier(learning_rate = 0.06, max_depth=12, min_data_in_leaf = 1300)

In [13]:
pipeline = Pipeline([('mdl', model)])

pipeline.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=1300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1300


Pipeline(steps=[('mdl',
                 LGBMClassifier(learning_rate=0.06, max_depth=12,
                                min_data_in_leaf=1300))])

In [14]:
y_pred = pipeline.predict(X_test)
print(roc_auc_score(y_test, y_pred))

0.5001959832814795


In [15]:
from joblib import dump
dump(pipeline, '../model.joblib')

['../model.joblib']

### Checking on what the X columns look like:

In [16]:
X.columns

Index(['SK_ID_CURR', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_REGISTRATION',
       ...
       'PREV_PRODUCT_COMBINATION_CashXSellmiddle_MEAN',
       'PREV_PRODUCT_COMBINATION_POShouseholdwithinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POShouseholdwithoutinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSindustrywithinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSindustrywithoutinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSmobilewithinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSmobilewithoutinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSotherwithinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_POSotherswithoutinterest_MEAN',
       'PREV_PRODUCT_COMBINATION_nan_MEAN'],
      dtype='object', length=355)

In [17]:
X.loc[X.SK_ID_CURR == 100003].to_numpy().reshape(-1)

array([ 1.00003000e+05,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  2.70000000e+05,  1.29350250e+06,  3.54100000e-03,
       -1.67650000e+04, -1.18600000e+03, -2.91000000e+02,  1.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  2.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.10000000e+01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.28000000e+02,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  

In [18]:
row = X.loc[X.SK_ID_CURR == 100003]
pipeline.predict_proba(row)

array([[0.98166352, 0.01833648]])

In [19]:
pip list --format=freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from platform import python_version

print(python_version())

3.9.7


# Separer le clean data csv en 10 pour éviter la limite de taille de Github

In [21]:
for i, df in enumerate(np.array_split(data, 10)):
    df.to_csv(f"clean_data{i+1}.csv", index=False)